In [40]:
from neuralprophet import NeuralProphet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
def make_df(filename, ID, slicing = 0):
    df = pd.read_csv(filename)
    
    if slicing:
        l = len(df)
        df = df.head(slicing)
        print(f'sliced into {len(df)} out of {l} data.')
        
    df.rename(columns={'Date' : 'ds' , 'S' : 'y'}, inplace = True)
    
    df['ds'] = pd.to_datetime(df['ds'])
    df.set_index('ds', inplace = True)
    df_daily = df.resample('D').mean()
    
    df_daily['H'].fillna(method='ffill', inplace = True)
    df = df_daily.reset_index()
    df['ID'] = ID
    
    df = df[['ds','y','H','ID']]
    
    return df

In [45]:
def make_cp(df,ID):
    prev = -1
    changePoint=[]
    for i in df.iloc:
        now = i['H']
        if prev!=now:
            changePoint.append(i['ds'])
            prev = now
    return changePoint

In [46]:
# 298 기간

df1 = make_df('H-S_seq_1.csv',ID = 'data1')
df2 = make_df('H-S_seq_2.csv',ID = 'data2')
df3 = make_df('H-S_seq_3.csv',ID = 'data3')
df4 = make_df('H-S_seq_4.csv',ID = 'data4')
df5 = make_df('H-S_seq_5.csv',ID = 'data5')
df6 = make_df('H-S_seq_6.csv',ID = 'data6')
df7 = make_df('H-S_seq_7.csv',ID = 'data7')
df8 = make_df('H-S_seq_8.csv',ID = 'data8')
df1_cp = make_cp(df1)
df2_cp = make_cp(df2)
df3_cp = make_cp(df3)
df4_cp = make_cp(df4)
df5_cp = make_cp(df5)
df6_cp = make_cp(df6)
df7_cp = make_cp(df7)
df8_cp = make_cp(df8)


df = pd.concat((df1,df2,df4,df6,df7,df8))

pd.set_option('display.max_rows', None)


In [53]:
m = NeuralProphet(learning_rate=0.1, 
                  yearly_seasonality=False, 
                  weekly_seasonality=False, 
                  unknown_data_normalization=True, 
                  newer_samples_weight = 4,
                  n_forecasts=10,
                  drop_missing = True,
                  optimizer = 'AdamW',
                  changepoints=df1_cp
                  )

m = m.add_future_regressor(name='H', normalize=True)
# metrics = m.fit(df,freq='D')
m.fit(df1)


WARNING - (NP.forecaster.fit) - Changing n_forecasts to 1. Without lags, the forecast can be computed for any future time, independent of lagged values
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.664% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as D
INFO - (NP.forecaster.__handle_missing_data) - dropped 236 NAN row in 'y'
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
ERROR - (NP.df_utils.get_normalization_params) - Normalization True not defined.
ERROR - (NP.df_utils.get_normalization_params) - Normalization True not defined.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 742
WARNING - (NP.forecaster._train) - To plot live loss

In [48]:
def warm_start_params(m):
    res = {}
    for pname in ['k', 'm', 'sigma_obs']:
        if m.mcmc_samples == 0:
            res[pname] = m.params[pname][0][0]
        else:
            res[pname] = np.mean(m.params[pname])
    for pname in ['delta', 'beta']:
        if m.mcmc_samples == 0:
            res[pname] = m.params[pname][0]
        else:
            res[pname] = np.mean(m.params[pname], axis=0)
    return res

In [49]:
fit_list = [df2,df4,df6,df7,df8]
cp_list = [df2_cp, df4_cp, df6_cp, df7_cp, df8_cp]

for i in range(5):
    m = NeuralProphet(learning_rate=0.1,
                      yearly_seasonality=False,
                      weekly_seasonality=False,
                      unknown_data_normalization=True,
                      newer_samples_weight=4,
                      n_forecasts=10,
                      drop_missing=True,
                      optimizer='AdamW',
                      changepoints=cp_list[i])
    m = m.add_future_regressor(name='H', normalize=True)
    # metrics = m.fit(df,freq='D')
    m.fit(fit_list[i], init = warm_start_params(pm))
    pm = m
    

AttributeError: 'NeuralProphet' object has no attribute 'mcmc_samples'